In [10]:
import imp
import numpy as np
import pandas as pd
import sagemaker

from pathlib import Path

imp.load_source('config', './config.py')
import config

In [18]:
# Read test dataset
pd_test = pd.read_csv(Path(config.input_path) / 'test.csv', index_col='id')
print(pd_test.shape)
display(pd_test.head(1))

(200000, 30)


,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10
id,,,,,,,,,,,,,,,,,,,,,
5,A,F,A,A,F,BI,A,AH,AX,A,...,0.73569,0.578366,0.723154,0.228037,0.356227,0.551249,0.655693,0.598331,0.359987,0.947489


In [12]:
# Read submission dataset
pd_submission = pd.read_csv(Path(config.input_path) / 'sample_submission.csv', index_col='id')
display(pd_submission.head(1))

,target
id,
5,0.5


# Inference

In [13]:
# Create predictor
predictor = sagemaker.predictor.Predictor(
    endpoint_name=config.endpoint_name,
    serializer=sagemaker.serializers.CSVSerializer(),
    deserializer=sagemaker.deserializers.CSVDeserializer())

In [57]:
# Inference. Batch processing performed due to SageMaker Endpoint input limits
output = []
step_size = 20000
for i in np.arange(0, 200000, step_size):
    X = pd_test.iloc[i:i+step_size, :].values
    response = predictor.predict(X)
    output.extend(response)

In [63]:
scores = [float(y[0]) for y in output]

# Predictions

In [68]:
pd_submission['target'] = scores
pd_submission.to_csv('./submission.csv')

# Clean Up

In [ ]:
predictor.delete_endpoint(delete_endpoint_config=True)